<a href="https://colab.research.google.com/github/lakshyasharma14/bro-code/blob/master/public/static/data/set_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Generic Set Predictor

Imports

In [18]:
from __future__ import absolute_import, division, print_function, unicode_literals

import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation
from keras.optimizers import gradient_descent_v2,adam_v2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import chain, combinations
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json

from google.colab import files
import csv
import random

Google Authentication to access the data sheet

In [19]:
from google.colab import auth
import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
auth.authenticate_user()

Get shopping cart data

In [20]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1bRsu9oH3N5A6N7rGFPZ3pngCvwfuHw4_57V_YCHUj38/edit?usp=sharing')

#sheet = wb.worksheet('encoded_filter_criteria')
sheet = wb.worksheet('shopping_cart')
raw_data = sheet.get_all_values()
df = pd.DataFrame(raw_data)

Tokenizing Data

In [21]:
data=[]
for inner in raw_data:
  tmp = []
  for str in inner:
    if str != '':
      tmp.append(str)
  data.append(tmp)

tok_data = {y for x in data for y in x}

tokenizer = Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n',split='#')
#tokenizer = Tokenizer()
tokenizer.fit_on_texts(tok_data)

sequences =tokenizer.texts_to_sequences(data)
random.shuffle(sequences)
sequences = list(filter(None, sequences))

Preparing data for training

In [22]:
FEATURE_SET_SIZE = max(list(map(max, *sequences)))+2
FEATURE_SET_SIZE

182

In [23]:
def binarize(num):
  dinarized_list = np.zeros(FEATURE_SET_SIZE, dtype=int)
  dinarized_list[num] = 1
  return dinarized_list;

def subset_list (num):
    s = range(num)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)))

def prepare_data(lst):
  result = np.zeros(FEATURE_SET_SIZE, dtype=int)
  for index in lst:
    result[index] = 1
  return result

def split_list(a_list):
    part = int(len(a_list)//3)
    return a_list[part:], a_list[:part]


In [24]:
input= []
output=[]
for row in sequences:
  row_len = len(row)
  for subset in subset_list(row_len):
    input_sum = np.zeros(FEATURE_SET_SIZE, dtype=int)
    output_sum = np.zeros(FEATURE_SET_SIZE, dtype=int)
    for index in range(row_len):
      if index in subset:
        input_sum =  [a + b for a, b in zip(input_sum, binarize(row[index]))]
      else:
        output_sum =  [a + b for a, b in zip(output_sum, binarize(row[index]))]
    input.append(input_sum)
    output.append(output_sum)

input= np.array(input)
output = np.array(output)

In [27]:
input, test_input= split_list(input)
output, test_output= split_list(output)

142460

Training model

In [29]:
model = Sequential()

model.add(Dense(FEATURE_SET_SIZE*2, activation='relu', input_dim=FEATURE_SET_SIZE))
model.add(Dropout(0.5))
model.add(Dense(FEATURE_SET_SIZE*2, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(FEATURE_SET_SIZE, activation=keras.activations.hard_sigmoid))

In [30]:
m = keras.metrics.SparseTopKCategoricalAccuracy(k=3)
model.compile(loss='binary_crossentropy',optimizer=adam_v2.Adam(learning_rate=0.001), metrics=["binary_accuracy"])

In [31]:
model.fit(input,output,epochs=1,batch_size=50)

2850/2850 [==============================] - 26s 9ms/step - loss: 0.1432 - binary_accuracy: 0.9818


Testing model


In [32]:
score= model.evaluate(test_input,test_output,batch_size=50)

1425/1425 [==============================] - 5s 4ms/step - loss: 0.1138 - binary_accuracy: 0.9829


In [33]:
score[1]*100

98.29342365264893

Time to test, with custom input and predict top m most popular items a person can buy if he buy x set of item


In [50]:
x=["soda"]
m=3
test_list = tokenizer.texts_to_sequences(x)
tmp_input=[]
tmp_input.append(prepare_data(test_list))
tmp_input.append(prepare_data(([1])))
tmp_input= np.array(tmp_input)
result = model.predict(tmp_input,batch_size=None,verbose=0,steps=None)
indices = (-result[0]).argsort()[:]
predicted_token = [x for x in indices if x not in test_list and x!= 0][:m]
final_prediction = tokenizer.sequences_to_texts([predicted_token])
final_prediction

['pastry canned beer whole milk']

In [57]:
x=["soda", "canned beer"]
m=3
test_list = tokenizer.texts_to_sequences(x)
tmp_input=[]
tmp_input.append(prepare_data(test_list))
tmp_input.append(prepare_data(([1])))
tmp_input= np.array(tmp_input)
result = model.predict(tmp_input,batch_size=None,verbose=0,steps=None)
indices = (-result[0]).argsort()[:]
predicted_token = [x for x in indices if x not in test_list and x!= 0][:m]
final_prediction = tokenizer.sequences_to_texts([predicted_token])
final_prediction

['specialty chocolate sausage frankfurter']

Save model for future!

In [59]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
files.download('model.json')

Saved model to disk


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>